In [47]:
import numpy as np
from astropy.io import fits
import sep
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from scipy import ndimage
import os

# Replace with downloaded file paths
f105w = 'hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits'
f125w = 'hlsp_hudf12_hst_wfc3ir_udfmain_f125w_v1.0_drz.fits'
f160w = 'hlsp_hudf12_hst_wfc3ir_udfmain_f160w_v1.0_drz.fits'

# Read images
d105 = fits.open(f105w)[0].data.astype(float)
d125 = fits.open(f125w)[0].data.astype(float)
d160 = fits.open(f160w)[0].data.astype(float)

# Simple stretches - use asinh stretch for each channel
def asinh_stretch(arr, scale=0.03):
    arr = arr - np.nanmin(arr)
    return np.arcsinh(arr * scale) / np.arcsinh(np.max(arr) * scale)

R = asinh_stretch(d160)
G = asinh_stretch(d125)
B = asinh_stretch(d105)

# normalize to 0..1:
def norm(x):
    x = np.nan_to_num(x)
    x -= x.min()
    if x.max() > 0:
        x /= x.max()
    return x

rgb = np.dstack([norm(R), norm(G), norm(B)])
plt.figure(figsize=(10,10))
plt.imshow(rgb, origin='lower')
plt.axis('off')
plt.savefig(os.path.join(outdir, 'udf_rgb_f160_R_f125_G_f105_B.png'), dpi=200, bbox_inches='tight')
plt.close()


In [48]:
# path to the f105w file you attached
f105w_path = 'hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits'
outdir = 'sep_udf_f105w_outputs'
os.makedirs(outdir, exist_ok=True)

hdul = fits.open(f105w_path)
data = hdul[0].data.astype('float32')
hdul.close()

# background
bkg = sep.Background(data)
data_sub = data - bkg

# detect
thresh = 1.5
objects = sep.extract(data_sub, thresh, err=bkg.globalrms)

# aperture photometry
r = 3.0
err = bkg.rms()

fluxes = []
flux_errs = []

for obj in objects:
    x = np.array([obj['x']])
    y = np.array([obj['y']])
    flux, fluxerr, flag = sep.sum_circle(data_sub, x, y, r, err=err)
    fluxes.append(flux[0])
    flux_errs.append(fluxerr[0])

fluxes = np.array(fluxes)
flux_errs = np.array(flux_errs)


# save figures as in Notebook 1 (omitted here for brevity — copy the plotting code from Notebook 1)


In [49]:
mean_flux = np.mean(fluxes)
median_flux = np.median(fluxes)
std_flux = np.std(fluxes)
max_idx = np.argmax(fluxes)
max_flux = fluxes[max_idx]
max_xy = (objects[max_idx]['x'], objects[max_idx]['y'])
sigma_away = (max_flux - mean_flux) / std_flux if std_flux > 0 else np.nan

print("Num detected:", len(objects))
print("Mean flux:", mean_flux)
print("Median flux:", median_flux)
print("Std flux:", std_flux)
print("Max flux:", max_flux, "at", max_xy, f"({sigma_away:.2f} sigma above mean)")


Num detected: 8642
Mean flux: 0.36178142855483114
Median flux: 0.030956558069156016
Std flux: 9.242459697958994
Max flux: 807.2972835731507 at (np.float64(1914.2549094883857), np.float64(1134.3164850742164)) (87.31 sigma above mean)
